In [0]:
import sys
sys.path.append("../../")

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

from src_staging.sv_customer_stage import create_customer_stage, hub_customer_schema

In [0]:
partition_name = 'partition_date'
partition_date = '20250828'

In [0]:
_bz_table = 'ctl_central_published.sc_bz_source_name.dt_customer_journey_daily'
_hub_sv_table = 'ctl_central_published.sc_sv_raw_vault.hub_customer_daily'
partition_date_fmt = f"{partition_date[:4]}-{partition_date[4:6]}-{partition_date[6:]}"

In [0]:
sv_df = create_customer_stage(_bz_table, partition_name, partition_date_fmt)
sv_df.display()

In [0]:
if not spark.catalog.tableExists(_hub_sv_table):
    hub_customer_schema = hub_customer_schema()
    (
        spark.createDataFrame([], hub_customer_schema)
        .write
        .format("delta")
        .partitionBy(partition_name)
        .saveAsTable(_hub_sv_table)
    )

In [0]:
incoming_data = sv_df.select(
                        col('customer_id'),
                        col('record_source'),
                        col('partition_date')
                    )\
                    .dropDuplicates(["customer_id"])
incoming_data.display()

In [0]:
join_table = incoming_data.join(spark.table(_hub_sv_table).select("customer_id"),
                            "customer_id", "left_anti")\
                            .withColumn("_ingest_ts", current_timestamp())
join_table.display()

In [0]:
(join_table.write
   .format("delta")
   .mode("append")
   .saveAsTable(_hub_sv_table))